In [24]:
from collections import deque, defaultdict
from typing import List, Tuple, Dict, Set
import random

# =========================
# 基本类型与参数
# =========================
Coord = Tuple[int, int]  # (r, c)
Grid  = List[List[int]]  # 0=过道, 1=货架

GRID_H, GRID_W = 10, 10
MAX_ITEMS_PER_SHELF = 4

# =========================
# 网格/货架工具
# =========================
DIRS = [(1,0),(-1,0),(0,1),(0,-1)]

def in_bounds(r: int, c: int, H: int, W: int) -> bool:
    return 0 <= r < H and 0 <= c < W

def build_grid_from_shelves(H: int, W: int, shelves: List[Coord]) -> Grid:
    """根据货架坐标生成网格：0=过道，1=货架。"""
    g = [[0]*W for _ in range(H)]
    for r, c in shelves:
        if not in_bounds(r, c, H, W):
            raise ValueError(f"Shelf {(r,c)} out of bounds for {H}x{W} grid.")
        g[r][c] = 1
    return g

def is_aisle(grid: Grid, rc: Coord) -> bool:
    r, c = rc
    return grid[r][c] == 0

def neighbors4(grid: Grid, rc: Coord):
    r, c = rc
    H, W = len(grid), len(grid[0])
    for dr, dc in DIRS:
        nr, nc = r+dr, c+dc
        if in_bounds(nr, nc, H, W):
            yield (nr, nc)

def any_adjacent_aisles(grid: Grid, shelf: Coord) -> Set[Coord]:
    """给定货架格，返回所有与之相邻的可达过道格。"""
    out = set()
    for nb in neighbors4(grid, shelf):
        if is_aisle(grid, nb):
            out.add(nb)
    return out

def reachable_aisles_from_start(grid: Grid, start: Coord) -> Set[Coord]:
    """在过道上 BFS，返回从 start 可达的所有过道格。"""
    if not is_aisle(grid, start):
        return set()
    q = deque([start])
    vis = {start}
    while q:
        u = q.popleft()
        for v in neighbors4(grid, u):
            if is_aisle(grid, v) and v not in vis:
                vis.add(v)
                q.append(v)
    return vis

# =========================
# 货架容量校验 & 随机摆放
# =========================
def validate_shelf_capacity(item_to_shelf: Dict[str, Coord], max_per_shelf: int = MAX_ITEMS_PER_SHELF):
    """检查每个货架的商品数量是否超上限。若超出，抛出异常。"""
    shelf_counts = defaultdict(list)
    for item, shelf in item_to_shelf.items():
        shelf_counts[shelf].append(item)
    overfilled = {s: items for s, items in shelf_counts.items() if len(items) > max_per_shelf}
    if overfilled:
        msg = ["Shelf capacity exceeded:"]
        for s, items in overfilled.items():
            msg.append(f"  Shelf {s} has {len(items)} (> {max_per_shelf}): {items}")
        raise ValueError("\n".join(msg))

def assign_items_to_shelves_random(
    item_names: List[str],
    shelves: List[Coord],
    max_per_shelf: int = MAX_ITEMS_PER_SHELF,
    seed: int = 42
) -> Dict[str, Coord]:
    """
    将所有商品随机分配到“手动输入”的货架列表中（每个货架最多 max_per_shelf 件商品）。
    返回: item_name -> shelf坐标
    """
    random.seed(seed)
    num_items = len(item_names)
    max_capacity = len(shelves) * max_per_shelf
    if num_items > max_capacity:
        raise ValueError(f"商品数量 {num_items} 超过总货架容量 {max_capacity}，请增加货架或减少商品。")

    shuffled = item_names[:]
    random.shuffle(shuffled)

    # 顺序填充
    shelf_loads: Dict[Coord, int] = {s: 0 for s in shelves}
    item_to_shelf: Dict[str, Coord] = {}

    for name in shuffled:
        # 找到仍有空位的货架，随机挑一个
        candidates = [s for s, cnt in shelf_loads.items() if cnt < max_per_shelf]
        if not candidates:
            raise ValueError("没有可用货架（容量已满）。")
        s = random.choice(candidates)
        shelf_loads[s] += 1
        item_to_shelf[name] = s

    # 最终容量校验（稳妥）
    validate_shelf_capacity(item_to_shelf, max_per_shelf)
    return item_to_shelf

# =========================
# 最短路径覆盖目标货架（核心评价）
# =========================
def shortest_path_cover_shelves(
    grid: Grid,
    start: Coord,
    basket_items: List[str],
    item_to_shelf: Dict[str, Coord],
) -> Dict[str, object]:
    """
    只走过道，访问所有目标货架（到达与之相邻的过道格即可视为取到）的最短步数。
    状态空间 BFS：状态=(过道位置, visited_mask)，边权=1（四邻步进）。
    """
    H, W = len(grid), len(grid[0])
    r0, c0 = start
    if not in_bounds(r0, c0, H, W):
        return dict(ok=False, reason="Start out of bounds.")
    if not is_aisle(grid, start):
        return dict(ok=False, reason="Start must be on an aisle (0).")

    # 目标货架去重
    shelves_needed: List[Coord] = []
    seen = set()
    for item in basket_items:
        if item not in item_to_shelf:
            return dict(ok=False, reason=f"Item '{item}' not in item_to_shelf mapping.")
        s = item_to_shelf[item]
        if s not in seen:
            seen.add(s)
            shelves_needed.append(s)

    # 每个目标货架的可取货位（与之相邻的过道格）
    pickup_positions: List[Set[Coord]] = []
    for s in shelves_needed:
        adj = any_adjacent_aisles(grid, s)
        pickup_positions.append(adj)

    # 起点过道连通性检查 + 可达性检查
    reachable_aisles = reachable_aisles_from_start(grid, start)
    unreachable = []
    for s, picks in zip(shelves_needed, pickup_positions):
        if not picks or not any(p in reachable_aisles for p in picks):
            unreachable.append(s)
    if unreachable:
        return dict(ok=False, reason="Some shelves are unreachable from start via aisles.", unreachable=unreachable)

    # 建立过道格 -> 目标bitmask 的映射（走到此过道即可“收集”该货架商品）
    aisle_pick_mask: Dict[Coord, int] = defaultdict(int)
    for idx, picks in enumerate(pickup_positions):
        bit = (1 << idx)
        for cell in picks:
            aisle_pick_mask[cell] |= bit

    # BFS on (pos, visited_mask)
    start_mask = aisle_pick_mask.get(start, 0)
    full_mask = (1 << len(shelves_needed)) - 1

    q = deque()
    q.append((start, start_mask))
    dist = {(start, start_mask): 0}

    while q:
        (r, c), mask = q.popleft()
        if mask == full_mask:
            steps = dist[((r, c), mask)]
            return dict(
                ok=True,
                steps=steps,
                avg_steps_per_item = steps / max(len(basket_items), 1),
                visited_shelves=shelves_needed
            )
        for nr, nc in neighbors4(grid, (r, c)):
            if not is_aisle(grid, (nr, nc)):
                continue
            new_mask = mask | aisle_pick_mask.get((nr, nc), 0)
            state = ((nr, nc), new_mask)
            if state not in dist:
                dist[state] = dist[((r, c), mask)] + 1
                q.append(state)

    return dict(ok=False, reason="No feasible path found.")

def evaluate_layout(
    grid: Grid,
    start: Coord,
    basket_items: List[str],
    item_to_shelf: Dict[str, Coord],
) -> Dict[str, object]:
    """
    对给定布局进行评价：输出最短总步数、平均每件商品步数。
    接口保持一致：grid/start/basket_items/item_to_shelf
    """
    # 容量强约束（评估前再次校验）
    validate_shelf_capacity(item_to_shelf, MAX_ITEMS_PER_SHELF)
    res = shortest_path_cover_shelves(grid, start, basket_items, item_to_shelf)
    return res

# =========================
# —— 配置区（手动输入）——
# =========================
if __name__ == "__main__":
    # 1) 手动输入：货架坐标（示例）
    #    你可以把 SHELVES 替换成你们的真实划分（保证“所有货架均可达”，即至少有一侧邻接过道格）
    SHELVES: List[Coord] = [
        (2,2),(2,3),(2,4),(3,4), (5,5),(6,5),(7,5),(3,7),
        (6,2),(6,3),(1,8),(2,8),(3,8),(8,9),(8,8),(1,1),(1,2),
        (1,3),(1,4),(1,5),(4,3),(4,4),(4,9),(5,2),(5,4),(6,6),(6,8),
        (3,1),(4,1),(5,1),(6,1)
    ]

    # 2) 手动输入：起点（必须在过道上）
    START: Coord = (0, 0)

    # 3) 商品清单（名称）——你可以换成你们的全集；本示例用少量名称演示
    ITEM_NAMES = [
        'almonds','antioxydant juice','asparagus','avocado','babies food','bacon','barbecue sauce',
        'black tea','blueberries','body spray','bramble','brownies','bug spray','burger sauce','burgers',
        'butter','cake','candy bars','carrots','cauliflower','cereals','champagne','chicken','chili',
        'chocolate','chocolate bread','chutney','cider','clothes accessories','cookies','cooking oil',
        'corn','cottage cheese','cream','dessert wine','eggplant','eggs','energy bar','energy drink',
        'escalope','extra dark chocolate','flax seed','french fries','french wine','fresh bread',
        'fresh tuna','fromage blanc','frozen smoothie','frozen vegetables','gluten free bar',
        'grated cheese','green beans','green grapes','green tea','ground beef','gums','ham',
        'hand protein bar','herb & pepper','honey','hot dogs','ketchup','light cream',
        'light mayo','low fat yogurt','magazines','mashed potato','mayonnaise','meatballs',
        'melons','milk','mineral water','mint','mint green tea','muffins','mushroom cream sauce',
        'napkins','nonfat milk','oatmeal','oil','olive oil','pancakes','parmesan cheese','pasta',
        'pepper','pet food','pickles','protein bar','red wine','rice','salad','salmon','salt',
        'sandwich','shallot','shampoo','shrimp','soda','soup','spaghetti','sparkling water',
        'spinach','strawberries','strong cheese','tea','tomato juice','tomato sauce',
        'tomatoes','toothpaste','turkey','vegetables mix','water spray','white wine',
        'whole weat flour','whole wheat pasta','whole wheat rice','yams','yogurt cake','zucchini'
    ]

    # 4) 购物篮（手动输入/替换为你们的真实篮子）
    BASKET = ['milk','bread']  # ← 演示：若与 ITEM_NAMES 不一致会提示映射缺失
    # 为了能跑通，这里改成从 ITEM_NAMES 中选（真实评估时请用你们的篮子）
    if any(x not in ITEM_NAMES for x in BASKET):
        BASKET = ITEM_NAMES[:5]  # demo：用前 5 个在清单中的商品

    # =========================
    # —— 流程：构建网格 → 随机摆放 → 评估
    # =========================
    grid = build_grid_from_shelves(GRID_H, GRID_W, SHELVES)

    # 随机摆放（容量约束，超出会报错）
    item_to_shelf = assign_items_to_shelves_random(
        item_names=ITEM_NAMES,
        shelves=SHELVES,
        max_per_shelf=MAX_ITEMS_PER_SHELF,
        seed=42
    )

    # 评估（只走过道，遇到目标货架相邻过道格即视为取到）
    result = evaluate_layout(
        grid=grid,
        start=START,
        basket_items=BASKET,
        item_to_shelf=item_to_shelf
    )

    print("\n=== Layout Evaluation ===")
    if result.get('ok'):
        print(f"Shortest steps: {result['steps']}")
        print(f"Avg steps per item: {result['avg_steps_per_item']:.2f}")
        print(f"Shelves visited (unique): {result['visited_shelves']}")
    else:
        print("Evaluation failed.")
        print("Reason:", result.get('reason'))
        if 'unreachable' in result:
            print("Unreachable shelves:", result['unreachable'])



=== Layout Evaluation ===
Evaluation failed.
Reason: Some shelves are unreachable from start via aisles.
Unreachable shelves: [(5, 2)]
